In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [4]:
import numpy as np
# np.save('/gdrive/My Drive/global_images_1024.npy', Global_images_array)
# np.save('/gdrive/My Drive/local_images_512.npy', Local_images_array)

Local_images_array = np.load('/gdrive/My Drive/local_images_512.npy')
Global_images_array = np.load('/gdrive/My Drive/global_images_1024.npy')

print(Local_images_array.shape)
# print(Global_images_array.shape)

(3200, 512, 512, 3)


#Resize Global

In [5]:
import cv2

Global_images_array1 = []

for i in range(len(Global_images_array)):
    # Load the image
    img = Global_images_array[i]
    
    # Resize the image to 512x512 using OpenCV
    resized_img = cv2.resize(img, (512, 512))
    
    # Replace the original image in the array with the resized image
    Global_images_array1.append(resized_img)

In [6]:
Global_images_array = np.array(Global_images_array1)

In [7]:
np.shape(Global_images_array)

(3200, 512, 512, 3)

In [8]:
import numpy as np

b_label = [0] * 800
n_label = [1] * 800
inv_label = [2] * 800
ins_label = [3] * 800
y = b_label + n_label + inv_label + ins_label

num_classes = 4


In [9]:
from keras.utils import to_categorical
labels = to_categorical(y)

In [ ]:
# from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np

# Load pre-trained ResNet50 model without top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# Create new model with custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in base model for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
import numpy as np

def split_data(X, y, test_size=0.2, random_state=5432):
    # Reshape X and y so that every 8 samples are treated as one datum
    n_datapoints = X.shape[0] // 8
    print(n_datapoints)
    X = X[:n_datapoints*8].reshape(n_datapoints, 8, X.shape[1], X.shape[2], X.shape[3])
    # X = X[:n_datapoints*8].reshape(n_datapoints, 8, X.shape[1])
    y = y[:n_datapoints*8].reshape(n_datapoints, 8, y.shape[1])
    
    # Split the data into training and testing sets
    n_test = int(n_datapoints * test_size)
    np.random.seed(random_state)
    indices = np.random.permutation(n_datapoints)
    X_train = X[indices[n_test:], :]
    y_train = y[indices[n_test:], :, :]
    X_test = X[indices[:n_test], :]
    y_test = y[indices[:n_test], :, :]
    
    return X_train, y_train, X_test, y_test

In [10]:
import numpy as np

def split_data_local_global(X,X1, y, test_size=0.2, random_state=5432):
    # Reshape X and y so that every 8 samples are treated as one datum
    n_datapoints = X.shape[0] // 8
    print(n_datapoints)
    X = X[:n_datapoints*8].reshape(n_datapoints, 8, X.shape[1], X.shape[2], X.shape[3])
    X1 = X1[:n_datapoints*8].reshape(n_datapoints, 8, X1.shape[1], X1.shape[2], X1.shape[3])
    # X = X[:n_datapoints*8].reshape(n_datapoints, 8, X.shape[1])
    y = y[:n_datapoints*8].reshape(n_datapoints, 8, y.shape[1])
    
    # Split the data into training and testing sets
    n_test = int(n_datapoints * test_size)
    np.random.seed(random_state)
    indices = np.random.permutation(n_datapoints)
    X_train = X[indices[n_test:], :]
    X_train_global = X1[indices[n_test:], :]
    y_train = y[indices[n_test:], :, :]
    X_test = X[indices[:n_test], :]
    X_test_global = X1[indices[:n_test], :]
    y_test = y[indices[:n_test], :, :]
    
    return X_train, X_train_global, y_train, X_test, X_test_global, y_test

In [ ]:
X_train, y_train, X_test, y_test = split_data(Local_images_array, labels)
X_train = X_train.reshape((320*8,512, 512, 3))
y_train = y_train.reshape((320*8,4))
X_test = X_test.reshape((80*8,512, 512, 3))
y_test = y_test.reshape((80*8,4))

400


In [ ]:
model.fit(X_train, y_train, epochs=8, batch_size=16)

Epoch 1/8
160/160 [==============================] - 9s 33ms/step - loss: 1.0015 - accuracy: 0.6062
Epoch 2/8
160/160 [==============================] - 5s 33ms/step - loss: 0.5027 - accuracy: 0.8113
Epoch 3/8
160/160 [==============================] - 5s 32ms/step - loss: 0.3870 - accuracy: 0.8633
Epoch 4/8
160/160 [==============================] - 5s 33ms/step - loss: 0.2968 - accuracy: 0.8918
Epoch 5/8
160/160 [==============================] - 5s 33ms/step - loss: 0.1987 - accuracy: 0.9320
Epoch 6/8
160/160 [==============================] - 5s 33ms/step - loss: 0.1080 - accuracy: 0.9703
Epoch 7/8
160/160 [==============================] - 5s 33ms/step - loss: 0.1098 - accuracy: 0.9680
Epoch 8/8
160/160 [==============================] - 5s 33ms/step - loss: 0.0558 - accuracy: 0.9902


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

20/20 [==============================] - 1s 61ms/step - loss: 0.5319 - accuracy: 0.8547
Test loss: 0.5319217443466187
Test accuracy: 0.854687511920929


In [ ]:
 model.save("/gdrive/My Drive/Colab Notebooks/Mahdi_ResNet50_Model_Local_1.h5")

# Feature Extraction phase

In [ ]:
from keras.models import Model

feature_model = Model(inputs=model.input, outputs=model.layers[-2].output)
features_train = feature_model.predict(X_train)

80/80 [==============================] - 5s 56ms/step


In [ ]:
features_test = feature_model.predict(X_test)

20/20 [==============================] - 1s 56ms/step


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
# fit model no training data
Xgb_clf = XGBClassifier()
Xgb_clf.fit(features_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
XGB_y_preds = Xgb_clf.predict(features_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, XGB_y_preds))

              precision    recall  f1-score   support

           0       0.66      0.61      0.64       200
           1       0.63      0.55      0.59       160
           2       0.79      0.76      0.77       160
           3       0.76      0.60      0.67       120

   micro avg       0.70      0.63      0.67       640
   macro avg       0.71      0.63      0.67       640
weighted avg       0.70      0.63      0.66       640
 samples avg       0.60      0.63      0.61       640



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Feature Extractin for Local_global

In [12]:
from keras.models import load_model

model = load_model('/gdrive/My Drive/Colab Notebooks/Mahdi_ResNet50_Model_Local_1.h5')

In [ ]:
X_train, X_train_global, y_train, X_test, X_test_global, y_test = split_data_local_global(Local_images_array,Global_images_array, labels)
X_train_global = X_train_global.reshape((320*8,512, 512, 3))
X_train = X_train.reshape((320*8,512, 512, 3))
y_train = y_train.reshape((320*8,4))
X_test = X_test.reshape((80*8,512, 512, 3))
X_test_global = X_test_global.reshape((80*8,512, 512, 3))
y_test = y_test.reshape((80*8,4))

400


In [ ]:
from keras.models import Model

feature_model = Model(inputs=model.input, outputs=model.layers[-2].output)
features_train = feature_model.predict(X_train)
features_train_global = feature_model.predict(X_train_global)

In [ ]:
features_test = feature_model.predict(X_test)
features_test_global = feature_model.predict(X_test_global)

20/20 [==============================] - 1s 55ms/step


In [ ]:
X_concatenated_train = np.concatenate((features_train, features_train_global), axis=1)
X_concatenated_test= np.concatenate((features_test, features_test_global), axis=1)

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
# fit model no training data
Xgb_clf = XGBClassifier()
Xgb_clf.fit(X_concatenated_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
XGB_y_preds = Xgb_clf.predict(X_concatenated_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, XGB_y_preds))

              precision    recall  f1-score   support

           0       0.69      0.60      0.64       200
           1       0.71      0.59      0.65       160
           2       0.81      0.77      0.79       160
           3       0.75      0.61      0.67       120

   micro avg       0.74      0.64      0.69       640
   macro avg       0.74      0.64      0.69       640
weighted avg       0.74      0.64      0.69       640
 samples avg       0.61      0.64      0.62       640



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Improvement in accuracy using Global Patches